In [1]:
!wget "https://raw.githubusercontent.com/sumitpundir/Verification-of-Parkinson-s-Disease-through-Voice-Data/master/parkinsons.csv"

--2018-12-06 19:13:13--  https://raw.githubusercontent.com/sumitpundir/Verification-of-Parkinson-s-Disease-through-Voice-Data/master/parkinsons.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40697 (40K) [text/plain]
Saving to: ‘parkinsons.csv’

parkinsons.csv      100%[===================>]  39.74K  --.-KB/s    in 0.01s   

2018-12-06 19:13:13 (3.10 MB/s) - ‘parkinsons.csv’ saved [40697/40697]



In [0]:
from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Convolution2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.datasets import mnist
from keras import optimizers
from keras.optimizers import Adam
from keras import backend as K
from keras import initializers
from keras.layers import Conv2D
from keras.layers import MaxPooling2D

In [3]:
import numpy as np
import pandas as pd
from time import time
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd
from time import time
from sklearn.cross_validation import train_test_split


import matplotlib.pyplot as plt
from sklearn import datasets

par = pd.read_csv("parkinsons.csv")

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [4]:
par.shape

(195, 24)

In [0]:
n_patients = par.shape[0]   ##number of patients
n_features = par.shape[1]-1 ##number of features

In [0]:
n_parkinsons = par[par['status'] == 1].shape[0] ##persons with parkinson 
n_healthy = par[par['status'] == 0].shape[0]

In [7]:
print("total number of patients:", n_patients)
print("number of features:", n_features)
print("number of patients with parkinson:", n_parkinsons)
print("number of patients without parkinsons:", n_healthy)


total number of patients: 195
number of features: 23
number of patients with parkinson: 147
number of patients without parkinsons: 48


In [0]:
X = list(par.columns[1:17]) + list(par.columns[18:])
Y  = par.columns[17]

In [9]:
par.columns[0]

'name'

In [0]:
Xf = par[X]
Yf = par[Y]

In [0]:
Yf

In [0]:
from sklearn.utils import class_weight

In [0]:
x_train,x_test,y_train,y_test = train_test_split(Xf, Yf, test_size=0.2, random_state=1)

In [14]:
y_test = list(y_test)
y_train = list(y_train)
y_test[0]

1

In [0]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)

In [0]:
class_weights

array([2.05263158, 0.66101695])

In [0]:
class_weight = {0: 3.,
                1: 1.}

In [0]:
y_train

In [16]:
x_train.shape

(156, 22)

In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
scaler = StandardScaler()

In [19]:
scaler.fit(x_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [0]:
x_train_normal = scaler.transform(x_train)

In [0]:
def build_network(input_features=None):
    # first we specify an input layer, with a shape == features
    inputs = Input(shape=(input_features,), name="input")
    x = Dense(64, activation='relu', name="hidden1")(inputs)
    x = Dense(32, activation='relu', name="hidden4")(x)
    x = Dense(16, activation='relu', name="hidden5")(x)
    prediction = Dense(1, activation='sigmoid', name="final")(x)
    model = Model(inputs=inputs, outputs=prediction)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])
    return model

In [0]:
input_features = x_train_normal.shape[1]

In [128]:
x_train_normal.shape

(156, 22)

In [0]:
model = build_network(input_features=input_features)

In [130]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 22)                0         
_________________________________________________________________
hidden1 (Dense)              (None, 64)                1472      
_________________________________________________________________
hidden4 (Dense)              (None, 32)                2080      
_________________________________________________________________
hidden5 (Dense)              (None, 16)                528       
_________________________________________________________________
final (Dense)                (None, 1)                 17        
Total params: 4,097
Trainable params: 4,097
Non-trainable params: 0
_________________________________________________________________


In [131]:
model.fit(x=x_train_normal, y=y_train, batch_size=16, epochs=25, verbose=1, validation_data=(x_test, y_test),class_weight=class_weight)

Train on 156 samples, validate on 39 samples
Epoch 1/25
156/156 [==============================] - 2s 11ms/step - loss: 0.8378 - acc: 0.6538 - val_loss: 3.4915 - val_acc: 0.7436
Epoch 2/25
156/156 [==============================] - 0s 180us/step - loss: 0.7152 - acc: 0.7308 - val_loss: 3.3938 - val_acc: 0.7436
Epoch 3/25
156/156 [==============================] - 0s 167us/step - loss: 0.6422 - acc: 0.7564 - val_loss: 3.5909 - val_acc: 0.7436
Epoch 4/25
156/156 [==============================] - 0s 158us/step - loss: 0.5797 - acc: 0.7821 - val_loss: 3.5920 - val_acc: 0.7436
Epoch 5/25
156/156 [==============================] - 0s 165us/step - loss: 0.5296 - acc: 0.8333 - val_loss: 4.0878 - val_acc: 0.7436
Epoch 6/25
156/156 [==============================] - 0s 168us/step - loss: 0.4810 - acc: 0.8526 - val_loss: 4.0700 - val_acc: 0.7436
Epoch 7/25
156/156 [==============================] - 0s 166us/step - loss: 0.4400 - acc: 0.8526 - val_loss: 4.0878 - val_acc: 0.7436
Epoch 8/25
156/156

In [0]:
y_res = model.predict(x_test)

In [106]:
y_res.shape

(39, 1)

In [107]:
y_res

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)

In [80]:
y_test

[1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0]